# Compute Integrated Vapor Transport IVT from u-, v- fluxes

This notebook loads the sample data (the u- and v- components of vertically integrated vapor fluxes) and compute the their magnitude: IVT.

In production you can use `compute_ivt.py` in the `scripts` folder for this step.


## Definition


IVT is simply defined as:

$IVT = \sqrt{u^2 + v^2}$, in $kg/m/s$.

where:

* u: u-component of vertically integrated vapor fluxes, in kg/m/s.
* v: v-component of vertically integrated vapor fluxes, in kg/m/s.


## Input data

Some metadata of the sample data:

* Source: ERA-I Reanalysis data 
* Temporal resolution: 6-hourly instantaneous snapshots.
* Horizontal resolution: 0.75 x 0.75 degree latitude/longitude.
* Temporal domain: 1984-01-01 -- 1984-01-31
* Spatial domain: 10-80 N, 0-360 E.
* Data format: netCDF.
* Size of data: ~10 Mb each.

The sample was obtained from the ERA Interim data web interface at:
https://apps.ecmwf.int/datasets/data/interim-full-daily/levtype=sfc/. The variable name for `uflux` on the web interface is "Vertical integral of eastward water vapour flux", and `vflux` is "Vertical integral of northward water vapour flux". The temporal and spatial domain was set according to the metadata listed above.


## Steps


Execute the following code blocks in sequence.


## Results

It will read in the *uflux* and *vflux* files, print out their meta data, compute the IVT value.
If run successfully, a new netCDF data file (`ivt_s_6_1984_Jan.nc`) will be created, saving the computed IVT data.


In [ ]:
%matplotlib inline
import os
import cdms2 as cdms
import MV2 as MV
from ipart.utils import funcs

UFLUX_FILE=os.path.join('.', 'uflux_s_6_1984_Jan.nc')
VFLUX_FILE=os.path.join('.', 'vflux_s_6_1984_Jan.nc')
OUTPUTFILE=os.path.join('.', 'ivt_s_6_1984_Jan.nc')

if not os.path.exists(UFLUX_FILE):
    raise Exception('UFLUX_FILE not found. Please make sure you downloaded the data file and save to the same dir as the notebook file.')
if not os.path.exists(VFLUX_FILE):
    raise Exception('VFLUX_FILE not found. Please make sure you downloaded the data file and save to the same dir as the notebook file.')

In [ ]:
#-----------Read in data----------------------
uflux=funcs.readVar(UFLUX_FILE, 'p71.162') # 'p71.162' is the id code for u-flux
vflux=funcs.readVar(VFLUX_FILE, 'p72.162') # 'p72.162' is the id code for v-flux

#-----------Print meta data------------------
print('\n######## Info of uflux ########')
print(uflux.info())

In [ ]:
print('Latitude of uflux:')
print(uflux.getLatitude())

In [ ]:
print('Longitude of uflux:')
print(uflux.getLongitude())

In [ ]:
print('Time axis of uflux:')
print(uflux.getTime())

In [ ]:
print('######## Info of uflux ########')
print(vflux.info())

# Please note that your input data need to have proper latitude,
# longitude and time axes, like the uflux above.

In [ ]:
# compute ivt

ivt=MV.sqrt(uflux*uflux+vflux*vflux)
ivt.id='ivt'
ivt.long_name='integrated vapor transport (IVT)'
ivt.standard_name=ivt.long_name
ivt.title=ivt.long_name
ivt.units=getattr(uflux, 'units', '')

#--------Save------------------------------------
print('\n# Saving output to:\n',OUTPUTFILE)
fout=cdms.open(OUTPUTFILE,'w')
fout.write(ivt,typecode='f')
fout.close()

#------- Plot some figure -----------
import matplotlib.pyplot as plt
from ipart.utils import plot

figure=plt.figure(figsize=(7,10),dpi=100)
idx=100  # select the 101th time step from the beginning
time_str=uflux.getTime().asComponentTime()[idx]

plot_vars=[uflux[idx], vflux[idx], ivt[idx]]
titles=['U', 'V', 'IVT']

for ii, vii in enumerate(plot_vars):
    axii=figure.add_subplot(3,1,ii+1)
    iso=plot.Isofill(vii, 10, 1, 2)
    plot.plot2(vii, iso, axii,
            title='%s, %s' %(str(time_str), titles[ii]),
            legend='local',
            projection='cyl',
            fix_aspect=False)

figure.show()